# FIFA API: World Cup players/teams

#### Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles
import numpy as np
import requests
import json

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('stiles')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [5]:
today_str = pd.Timestamp("today").strftime("%m/%d/%Y")
today = pd.Timestamp("today")

---

## Get data

#### Read teams json from FIFA API

In [6]:
team_json = requests.get(
    "https://api.fifa.com/api/v3/competitions/teams/255711?language=en"
).json()["Results"]

#### Read into a dataframe and lowercase column names

In [7]:
teams_df = pd.DataFrame(team_json)

In [8]:
teams_df.columns = teams_df.columns.str.lower()

#### Deal with nested info

In [9]:
teams_df["team_name"] = pd.json_normalize(pd.json_normalize(teams_df["name"])[0])[
    "Description"
]

In [10]:
teams_df["team_abbr"] = pd.json_normalize(
    pd.json_normalize(teams_df["displayname"])[0]
)["Description"]

In [11]:
teams_df_slim = teams_df[
    [
        "idteam",
        "idconfederation",
        "shortclubname",
        "abbreviation",
    ]
].copy()

In [12]:
teams_df_slim["pictureurl"] = (
    "https://cloudinary.fifa.com/api/v3/picture/flags-sq-4/" + teams_df["abbreviation"]
)

In [13]:
teams_list = teams_df_slim["idteam"].to_list()

#### Create a dict to map team ids to full names

In [14]:
team_id_to_name = pd.Series(
    teams_df_slim.shortclubname.values, index=teams_df_slim.idteam
).to_dict()

---

#### Get player details from FIFA API

In [15]:
player_jsons = []
players_list = []

for t in teams_list:
    player_response = requests.get(
        f"https://api.fifa.com/api/v3/teams/{t}/squad?idCompetition=17&idSeason=255711&language=en"
    )
    for j in player_response.json()["Players"]:
        player_dict = {
            "player_id": j["IdPlayer"],
            "name": j["PlayerName"][0]["Description"],
            "weight": j["Weight"],
            "height": j["Height"],
            "country": j["IdCountry"],
            "country_id": j["IdTeam"],
            "number": j["JerseyNum"],
            "birthday": j["BirthDate"],
            "picture": j["PlayerPicture"]["PictureUrl"],
        }
        players_list.append(player_dict)

In [16]:
players_df = pd.DataFrame(players_list).sort_values("height", ascending=False)

#### Clean up a bit

In [17]:
players_df["number"] = players_df["number"].astype(str)

In [18]:
players_df["name"] = players_df["name"].str.title()

#### Ages

In [19]:
players_df["dob"] = pd.to_datetime(players_df["birthday"]).dt.strftime("%Y-%m-%d")

In [20]:
players_df["age"] = (
    ((pd.to_datetime(today_str) - pd.to_datetime(players_df["dob"])).dt.days) / 365.25
).round(2)

In [21]:
players_df = players_df.drop(["birthday"], axis=1)

#### Add country names

In [22]:
players_df["country_name"] = players_df["country_id"].map(team_id_to_name)

---

## Aggregate

#### Mean height, weight and age, by team

In [23]:
players_grouped = (
    players_df.groupby(["country_name"])
    .agg({"weight": "mean", "height": "mean", "age": "mean"})
    .reset_index()
)

#### Tallest

In [24]:
players_grouped.sort_values("height", ascending=False).head()

,country_name,weight,height,age
25,Serbia,78.461538,187.192308,27.378077
8,Denmark,82.000000,186.115385,27.697692
11,France,82.538462,185.461538,27.125385
12,Germany,80.038462,184.730769,27.338846
2,Belgium,78.461538,184.653846,28.363462


#### Oldest

In [25]:
players_grouped.sort_values("age", ascending=False).head()

,country_name,weight,height,age
14,IR Iran,79.000000,183.720000,29.436400
17,Mexico,74.115385,178.807692,28.983077
28,Tunisia,76.961538,182.692308,28.553846
3,Brazil,74.846154,180.769231,28.448846
2,Belgium,78.461538,184.653846,28.363462


---

## USA

In [26]:
usa_players_df = players_df.query('country == "USA"')

In [27]:
usa_players_df.head()

,player_id,name,weight,height,country,country_id,number,picture,dob,age,country_name
764,385917,Ethan Horvath,84.0,193.0,USA,43921,12,https://digitalhub.fifa.com/transform/f3b199e6...,1995-06-09,27.47,USA
771,390276,Haji Wright,79.0,191.0,USA,43921,19,https://digitalhub.fifa.com/transform/f1ee9fad...,1998-03-27,24.67,USA
753,448217,Matt Turner,79.0,190.0,USA,43921,1,https://digitalhub.fifa.com/transform/402cd2c6...,1994-06-24,28.42,USA
755,369097,Walker Zimmerman,84.0,190.0,USA,43921,3,https://digitalhub.fifa.com/transform/388db833...,1993-05-19,29.52,USA
777,305832,Sean Johnson,91.0,190.0,USA,43921,25,https://digitalhub.fifa.com/transform/61a88ddb...,1989-05-31,33.49,USA


---

#### No. 10s

In [28]:
players_df.query('number == "10"')

,player_id,name,weight,height,country,country_id,number,picture,dob,age,country_name
165,214876,Bryan Ruiz,75.0,187.0,CRC,43901,10,https://digitalhub.fifa.com/transform/839f9485...,1985-08-18,37.27,Costa Rica
528,269735,Grzegorz Krychowiak,83.0,186.0,POL,43962,10,https://digitalhub.fifa.com/transform/2cd38eae...,1990-01-29,32.82,Poland
373,330659,Karim Ansarifard,80.0,186.0,IRN,43817,10,https://digitalhub.fifa.com/transform/d0a1b57b...,1990-04-03,32.65,IR Iran
710,311558,Granit Xhaka,82.0,183.0,SUI,43971,10,https://digitalhub.fifa.com/transform/6e96ca24...,1992-09-27,30.16,Switzerland
736,364669,Wahbi Khazri,76.0,182.0,TUN,43888,10,https://digitalhub.fifa.com/transform/ac013426...,1991-02-08,31.80,Tunisia
217,321716,Christian Eriksen,76.0,181.0,DEN,43941,10,https://digitalhub.fifa.com/transform/e454f07e...,1992-02-14,30.78,Denmark
658,298698,Dusan Tadic,76.0,181.0,SRB,1902465,10,https://digitalhub.fifa.com/transform/d6f6b0ce...,1988-11-20,34.02,Serbia
35,404319,Ajdin Hrustic,70.0,180.0,AUS,43976,10,https://digitalhub.fifa.com/transform/1902b759...,1996-07-05,26.39,Australia
424,390525,Lee Jaesung,70.0,180.0,KOR,43822,10,https://digitalhub.fifa.com/transform/d922c344...,1992-08-10,30.29,Korea Republic
762,390267,Christian Pulisic,72.0,179.0,USA,43921,10,https://digitalhub.fifa.com/transform/4c947748...,1998-09-18,24.19,USA


In [29]:
players_df["picture"][266]

'https://digitalhub.fifa.com/transform/c5fc5a56-6169-418f-828a-f5482b36a21a/1441945587'

---

## Exports

In [30]:
players_df.to_csv("data/processed/world_cup_players.csv", index=False)
teams_df_slim.to_csv("data/processed/world_cup_teams.csv", index=False)
usa_players_df.to_csv("data/processed/world_cup_teams_usa.csv", index=False)
players_df.query('number == "10"').to_csv(
    "data/processed/world_cup_players_number10s.csv", index=False
)